## PART 1

In [95]:
with open('input.txt') as f:
    input = list(map(lambda x: x.strip(), f.readlines()))

PATHS = []

for l in input:

    pos, delta = l.split(' @ ') 
    x, y, _ = list(map(lambda x: int(x), pos.split(', ')))
    delta_x, delta_y, _ = list(map(lambda x: int(x), delta.split(', ')))

    m = delta_y / delta_x
    c = y - m*x

    PATHS.append([m,c,x,y,delta_x,delta_y])

res = 0
test_min, test_max = 200000000000000, 400000000000000

for i in range(len(PATHS)):
    for j in range(i+1, len(PATHS)):
        
        # parallel lines
        if (PATHS[j][0] - PATHS[i][0]) == 0:
            continue

        x_intersection = (PATHS[i][1] - PATHS[j][1]) / (PATHS[j][0] - PATHS[i][0])
        y_intersection = PATHS[i][0] * x_intersection + PATHS[i][1]
        
        
        # A incompatible
        if ((x_intersection > PATHS[i][2] and PATHS[i][4] < 0) or
            (x_intersection < PATHS[i][2] and PATHS[i][4] > 0) or
            (y_intersection > PATHS[i][3] and PATHS[i][5] < 0) or
            (y_intersection < PATHS[i][3] and PATHS[i][5] > 0)):
            continue
        
        # B incompatible
        if ((x_intersection > PATHS[j][2] and PATHS[j][4] < 0) or
            (x_intersection < PATHS[j][2] and PATHS[j][4] > 0) or
            (y_intersection > PATHS[j][3] and PATHS[j][5] < 0) or
            (y_intersection < PATHS[j][3] and PATHS[j][5] > 0)):
            continue
        
        # cross outside test area
        if not (test_min <= x_intersection <= test_max and 
                test_min <= y_intersection <= test_max):
            continue

        res += 1

res                        

11995

## PART 2

#### *IDEA rubata dal subreddit, shame on me*

To figure the correct starting point + velocity, we solve some systems of linear equations. 

Write your starting point and velocity as unknowns x, y, z, Vx, Vy, Vz. 

Equating the position of the rock at time t1 (another unknown parameter) with the position of one of the hailstones at the same time t1, we get a system of 3 equations and 7 unknowns. 

Unfortunately, these equations have degree 2 - this is not a linear system!

However, manipulating these equations a bit we can get a linear equation of the type:

(Vy1-Vy2)x - (Vx1-Vx2)y + - (y1-y2)Vx + (x1-x2)Vy = y2Vx2 + x2Vy2 - y1Vx1 - x1Vy1

Where x1, y1, z2, Vx1, Vy1, Vz1 and x2, y2, z2, Vx2, Vy2, Vz2 are the starting points and velocities of two of the hailstones.

So with 2 lines we can get a linear equation. Similarly, we can get equations involving the unknowns z and Vz.

We can use the myriad of hailstones we have to generate as many equations as we like. 

The system is going to be overdetermined, but the problem statement seems to ensure that there is going to be a solution. 

On the other hand it can happen that we make a bad choice of lines and the equation we use are underdetermined. 

This last problem is not accounted for in the code - if it happens, one can shuffle the input file until it works.

In [96]:
import numpy as np
from collections import namedtuple

In [97]:
Halestone = namedtuple('Halestone', ['x', 'y', 'z', 'Δx', 'Δy', 'Δz'])

H = []

for l in input[:6]:

    pos, delta = l.split(' @ ') 
    x, y, z = list(map(lambda x: int(x), pos.split(', ')))
    Δx, Δy, Δz = list(map(lambda x: int(x), delta.split(', ')))

    H.append(Halestone(x,y,z,Δx,Δy,Δz))

In [99]:
# ho provato ad usare un po' di Halestones diverse fino a che non ha funzionato LOL

# First equation: lines 0 and 1, x and y only
# Second equation: lines 0 and 4, x and y only
# Third equation: lines 0 and 1, x and z only
# Fourth equation: lines 0 and 4, x and z only
# Fifth equation: lines 0 and 1, y and z only
# Sixth equation: lines 0 and 4, y and z only

A = np.array([  [H[0].Δy - H[1].Δy, H[1].Δx - H[0].Δx, 0.0, H[1].y - H[0].y, H[0].x - H[1].x, 0.0],
                [H[0].Δy - H[4].Δy, H[4].Δx - H[0].Δx, 0.0, H[4].y - H[0].y, H[0].x - H[4].x, 0.0],
		        [H[0].Δz - H[1].Δz, 0.0, H[1].Δx - H[0].Δx, H[1].z - H[0].z, 0.0, H[0].x - H[1].x],
		        [H[0].Δz - H[4].Δz, 0.0, H[4].Δx - H[0].Δx, H[4].z - H[0].z, 0.0, H[0].x - H[4].x],
		        [0.0, H[0].Δz - H[1].Δz, H[1].Δy - H[0].Δy, 0.0, H[1].z - H[0].z, H[0].y - H[1].y],
		        [0.0, H[0].Δz - H[4].Δz, H[4].Δy - H[0].Δy, 0.0, H[4].z - H[0].z, H[0].y - H[4].y]  ])

b = np.array([  H[1].y * H[1].Δx - H[1].x * H[1].Δy - H[0].y * H[0].Δx + H[0].x * H[0].Δy,
		        H[4].y * H[4].Δx - H[4].x * H[4].Δy - H[0].y * H[0].Δx + H[0].x * H[0].Δy,
                H[1].z * H[1].Δx - H[1].x * H[1].Δz - H[0].z * H[0].Δx + H[0].x * H[0].Δz,
		        H[4].z * H[4].Δx - H[4].x * H[4].Δz - H[0].z * H[0].Δx + H[0].x * H[0].Δz,
		        H[1].z * H[1].Δy - H[1].y * H[1].Δz - H[0].z * H[0].Δy + H[0].y * H[0].Δz,
		        H[4].z * H[4].Δy - H[4].y * H[4].Δz - H[0].z * H[0].Δy + H[0].y * H[0].Δz   ])

x = np.linalg.solve(A, b)

int(x[0] + x[1] + x[2])

983620716335751